In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from time import time
from sklearn.metrics import mean_absolute_error
import wandb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def data():
    training = pd.read_csv("./training_preprocessed")
    validation = pd.read_csv("./validation_preprocessed")
    test = pd.read_csv("./test_preprocessed")
    X_train = training.drop(columns="y")
    y_train = training["y"]
    X_valid = validation.drop(columns="y")
    y_valid = validation["y"]
    X_test = test
    return X_train, y_train, X_valid, y_valid, X_test

In [3]:
X_train, y_train, X_valid, y_valid, X_test = data()

In [4]:
wandb.init(project="house-price-prediction", entity="luiscostigan")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: luiscostigan (use `wandb login --relogin` to force relogin)


In [23]:
def create_model():
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(wandb.config.dropout1))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='rmsprop', loss="mse", metrics=["mae"])
    
    return model

In [24]:
from sklearn.model_selection import RepeatedKFold

# Instantiate the validator, have it spit out the indices, generate batches from the indices, and feed those batches to the model.
rkf = RepeatedKFold(n_splits=5, n_repeats=2)
rkf.get_n_splits(X_train, y_train)

10

8/8 [==============================] - 0s 2ms/step - loss: 699128960.0000 - mae: 15479.1387


Currently trying to implement RepeatedKFold cross validation with the Keras Sequential model, while recording model results on WandB.

Sometimes I am getting a failed-1 error from WandB.
    - 

In [25]:
from wandb.keras import WandbCallback

sweep_config = {
  "name": "keras-sequential-model-sweep",
  "method": "random",
  "parameters": {
    "dropout1": {
      "min": 0.0,
      "max": 0.8
    },
    "epochs": {
      "values": [30,50,100]
    },
    "batch_size": {
      "values": [16, 64, 128, 512]
    }
  }
}

config_defaults = {
  "dropout1": 0.5,
  "epochs": 50,
  "batch_size": 64
}

def train():

  # Go through each split, and get the index number for each (currently 50, 10 x 5 repeated k-fold) 
  for index, (train_indices, val_indices) in enumerate(rkf.split(X_train, y_train)):
    print("Training on fold " + str(index+1) + "/10...")

    # With the current session in WandB
    with wandb.init(config=config_defaults) as run:

      # Generate batches from indices
      xtrain, xval = X_train.values[train_indices], X_train.values[val_indices]
      ytrain, yval = y_train[train_indices], y_train[val_indices]

      # Recreate the model each time for each new batch
      model = None # Not sure if this step is necessary
      model = create_model()

      # Fit model on new batches
      model.fit(
        np.asarray(xtrain), 
        ytrain, 
        epochs=wandb.config.epochs, 
        batch_size=wandb.config.batch_size, 
        verbose=0,
        callbacks=[WandbCallback()], 
        validation_data=(np.asarray(xval),yval)
        )
      
      # Generate data for each
      test_mse_score, test_mae_score = model.evaluate(np.asarray(xval), yval, callbacks=[WandbCallback()])

# Am I aggregating RepeatedKFold results properly? What is the correct process for this?

keras_sequential_sweep_1 = wandb.sweep(sweep_config, project="house-price-prediction", entity="luiscostigan")

count = 3
wandb.agent(keras_sequential_sweep_1, function=train, count=count)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: ayoa4ibu
Sweep URL: https://wandb.ai/luiscostigan/house-price-prediction/sweeps/ayoa4ibu


wandb: Agent Starting Run: yjha9zj2 with config:
wandb: 	batch_size: 64
wandb: 	dropout1: 0.5985095856959446
wandb: 	epochs: 100


Training on fold 1/10...


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,██▇▆▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mae,██▇▆▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▇▆▅▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mae,██▇▆▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_epoch,42
best_val_loss,673457856.0
epoch,49
loss,555146944.0
mae,13831.8291
val_loss,699129024.0


...

8/8 [==============================] - 0s 2ms/step - loss: 429295104.0000 - mae: 13455.7695


epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
loss,█▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mae,█▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mae,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_epoch,29
best_val_loss,429295104.0
epoch,29
loss,566095872.0
mae,14003.18164
val_loss,429295104.0


Training on fold 10/10...


8/8 [==============================] - 0s 2ms/step - loss: 501083520.0000 - mae: 15936.9824


epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
loss,█▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mae,█▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mae,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_epoch,26
best_val_loss,433571936.0
epoch,29
loss,580755520.0
mae,13737.4873
val_loss,501083392.0


Train the model with Hyperopt to find best parameters.
Retrain model with new best parameters.
Make predictions.

In [18]:
best_params = {
    "dropout": 0.2,
    "epochs": 100,
    "batch_size": 128
}

In [20]:
best_params = {
    "dropout": 0.2,
    "epochs": 100,
    "batch_size": 128
}

def make_predictions(best_params):
    
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(best_params.get("dropout")))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='rmsprop', loss="mse", metrics=["mae"])
              
    model.fit(X_train, y_train, epochs=best_params.get("epochs"),
              batch_size=best_params.get("batch_size"), verbose = 0)
    
    preds  = model.predict(X_test, best_params.get("batch_size"), verbose = 0)
              
    return preds

In [22]:
predictions = make_predictions(best_params)

## Visualize Model Output

In [26]:
d

{'Id': RangeIndex(start=0, stop=1459, step=1),
 'SalePrice': array([[136474.11],
        [156334.36],
        [188805.05],
        ...,
        [181121.44],
        [106913.19],
        [214941.02]], dtype=float32)}

In [29]:
d = {"Id":X_test.index,"SalePrice":predictions.flatten()}
submission = pd.DataFrame(data=d, index=None)

# Revert log transform
#submission["SalePrice"] = np.exp(submission["SalePrice"])

In [30]:
submission

,Id,SalePrice
0,0,136474.109375
1,1,156334.359375
2,2,188805.046875
3,3,195929.828125
4,4,200915.140625
...,...,...
1454,1454,76588.890625
1455,1455,63704.851562
1456,1456,181121.437500
1457,1457,106913.187500


In [31]:
submission.to_csv("submission.csv",index=False)